In [313]:
from pathlib import Path
import pandas as pd
from collections import defaultdict

In [314]:
input = Path("example.txt").read_text()
print(input)

47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47



In [315]:
raw_rules, raw_updates = input.split("\n\n")

In [316]:
updates = [
    [int(page) for page in update.split(",")] for update in raw_updates.splitlines()
]
updates

[[75, 47, 61, 53, 29],
 [97, 61, 53, 29, 13],
 [75, 29, 13],
 [75, 97, 47, 61, 53],
 [61, 13, 29],
 [97, 13, 75, 29, 47]]

In [317]:
rule_df = pd.DataFrame(
    [[rule for rule in rule_line.split("|")] for rule_line in raw_rules.splitlines()],
    columns=["First", "Second"],
    dtype="Int16",
)
rule_df

,First,Second
0,47,53
1,97,13
2,97,61
3,97,47
4,75,29
5,61,13
6,75,53
7,29,13
8,97,29
9,53,29


In [318]:
rules = rule_df.groupby("Second")["First"].apply(set).to_dict()
rules = defaultdict(set, rules)
rules

defaultdict(set,
            {13: {29, 47, 53, 61, 75, 97},
             29: {47, 53, 61, 75, 97},
             47: {75, 97},
             53: {47, 61, 75, 97},
             61: {47, 75, 97},
             75: {97}})

In [321]:
# solution 1


def find_middle(update: list) -> int:
    return update[len(update) // 2]


def find_broken_rules(pages: list, rule: set) -> set:
    return set(pages).intersection(rule)


def process(update):
    for i, page in enumerate(update):
        following_pages = update[i + 1 :]
        broken_rules = find_broken_rules(following_pages, rules[page])
        if len(broken_rules) > 0:
            return 0
    return find_middle(update)


sum([process(update) for update in updates])

143

In [329]:
# solution 2
def process(update):
    i = 0
    num_updates = len(update)
    has_broken_rule = False
    while i < num_updates:
        page = update[i]
        following_pages = update[i + 1 :]
        broken_rules = find_broken_rules(following_pages, rules[page])
        if len(broken_rules) == 0:
            i += 1
            continue
        
        has_broken_rule = True
        # Bubble sort?
        update.remove(page)
        for j, following_page in enumerate(following_pages[::-1]):
            if following_page in broken_rules:
                update.insert(len(update) - j, page)
                break
            
    return update[len(update) // 2] if has_broken_rule else 0


sum([process(update.copy()) for update in updates])

123